# Datos

http://www.santafe.gob.ar/ms/covid19/datosabiertos/

In [ ]:
from datetime import datetime

import pandas as pd

from plotly import express as px

pd.set_option('display.max_rows', 1000)

In [ ]:
casos = pd.read_csv("https://metabase.santafe.gob.ar/api/public/card/c5e1ebaf-77ce-402b-97ae-06498501b342/query/csv")
casos.tail()

In [ ]:
casos.rename(columns={col: col.lower().replace(" ", "_") for col in casos.columns}, inplace=True)
casos.rename(columns={f: f + "_accum" for f in ("confirmados", "descartados", "en_estudio", "notificaciones")},
             inplace=True)
casos.localidad = casos.localidad.str.capitalize()
casos.fecha = pd.to_datetime(casos.fecha)
casos.sort_values("fecha", inplace=True)

# agregar delta y medias moviles de casos por localidad
for localidad in casos.localidad.unique():
    index_localidad = casos.localidad == localidad
    casos_localidad = casos[index_localidad]
    
    confirmados_localidad = casos_localidad["confirmados_accum"] - casos_localidad["confirmados_accum"].shift(1)
    media_3_localidad = confirmados_localidad.rolling(3).mean()
    media_7_localidad = confirmados_localidad.rolling(7).mean()
    
    casos.loc[index_localidad, "confirmados"] = confirmados_localidad
    casos.loc[index_localidad, "confirmados_media_3"] = media_3_localidad
    casos.loc[index_localidad, "confirmados_media_7"] = media_7_localidad

casos.tail()

In [ ]:
casos.fecha.max()

In [ ]:
def graficar(casos, campo="confirmados_media_7", por_localidad=False, region=None):   
    if region is None:
        region = ', '.join(casos.localidad.unique())
    
    aclaracion = ""
    if "media" in campo:
        aclaracion = f" (media móvil de {campo.split('_')[-1]} días)"
    
    if por_localidad:
        casos = casos.groupby(["fecha", "localidad"]).sum().reset_index().set_index("fecha")
        color = "localidad"
    else:
        casos = casos.groupby("fecha").sum()
        color = None
   
    labels = {
        campo: "",
        "fecha": "",
    }

    fig = px.line(
        casos,
        y=campo, 
        title=f"Casos confirmados en {region}{aclaracion}",
        labels=labels,
        color=color,
    )

    fig.add_vline(
        x=datetime(2021, 2, 17), 
        line_width=2, 
        line_dash="dot", 
        line_color="green",
    )

    fig.add_vline(
        x=datetime(2021, 4, 4), 
        line_width=2, 
        line_dash="dot", 
        line_color="red",
    )
    
    return fig

In [ ]:
graficar(casos[casos.localidad == "Rafaela"], campo="confirmados_media_7")

In [ ]:
graficar(casos[casos.localidad.isin(("Rafaela", "Venado tuerto"))], campo="confirmados_media_7", por_localidad=True)

In [ ]:
graficar(casos, por_localidad=False, region="Provincia de Santa Fe")

In [ ]:
casos[casos.localidad == "Rafaela"].tail(15)